# HTMDEC DMS API Example

This example demonstrates how to use the new HTMDEC DMS `/form` and `/entry` REST API endpoints to search for IMQCAM data which can be converted to a Pandas dataframe for analysis and visualization. 

### Using the REST API

The [Girder Client](https://girder.readthedocs.io/en/latest/python-client.html) can be used to query the REST endpoints directly. When running via the DMS, the API URL and user token required for access are available in the current environment.


In [1]:
from girder_client import GirderClient
import os
import json
import pandas as pd

client = GirderClient(apiUrl='https://data.imqcam.org/api/v1')
client.authenticate(apiKey=os.environ['GIRDER_API_KEY'])

{'_id': '65c5387402ad536bd833de56'}

In [2]:
igsn = client.get('deposition', parameters={'limit': 1000})
print(f"Number of IGSNs generated so far: {len(igsn)}")

Number of IGSNs generated so far: 562


A variety of data has been collected, from powder synthesis or build details, to characterizations

In [3]:
forms = client.get('entry', parameters={'limit': 1000})
print("Total number of form entries: ", len(forms))

Total number of form entries:  592


3 build origins have been scoped:

In [4]:
imqcam_schema_id = '66425a71b18fa1c426e93aa0'
ttt_schema_id = '67d39472366ec49ab59dd4db'
uli_schema_id = '68922e35f5b193b7d3e07f5b'
imqcam_builds = [form for form in forms if form['formId'] == imqcam_schema_id]
print("Number of CMU IMQCAM build forms: ", len(imqcam_builds))
ttt_builds = [form for form in forms if form['formId'] == ttt_schema_id]
print("Number of TTT forms: ", len(ttt_builds))
uli_builds = [form for form in forms if form['formId'] == uli_schema_id]
print("Number of ULI forms: ", len(uli_builds))
print("Total number of builds: ", len(uli_builds) + len(ttt_builds) + len(imqcam_builds))

Number of CMU IMQCAM build forms:  3
Number of TTT forms:  201
Number of ULI forms:  192
Total number of builds:  396


# CMU Printer Forms

An example of an IMQCAM build entry:

In [5]:
imqcam_builds[0]

{'_id': '68228459f43924f82b944d33',
 'created': '2025-05-12T23:29:29.265000+00:00',
 'creatorId': '67ad6270703d68f56a59a4e7',
 'data': {'assignedIGSN': 'CMXMAL00007',
  'buildGeometries': [{'count': 30, 'geometryType': 'TEN'},
   {'count': 20, 'geometryType': 'AXFT'}],
  'buildMachine': 'EOS M290 at CMU',
  'buildParameters': {'contouringParameters': {'hatchSpacing': 0,
    'laserPower': 150,
    'laserVelocity': 1250,
    'layerThickness': 30},
   'downskinParameters': {'hatchSpacing': 0.08,
    'laserPower': 90,
    'laserVelocity': 1200,
    'layerThickness': 30},
   'infillParameters': {'hatchSpacing': 0.14,
    'laserPower': 350,
    'laserVelocity': 1300,
    'layerThickness': 30},
   'upskinParameters': {'hatchSpacing': 0,
    'laserPower': 0,
    'laserVelocity': 0,
    'layerThickness': 0}},
  'buildPlate': {'material': 'Ti-6Al-4V', 'temperature': 175},
  'eosReportFile': '',
  'extraInfo': [],
  'flowGas': {'flowRate': 0, 'o2Concentration': 0, 'type': 'Ar'},
  'igsn': {'batch

What we can infer from the IMQCAM builds:

In [6]:
from collections import defaultdict

# Part 1: Geometry counts per IGSN
igsn_geometry_counts = defaultdict(lambda: defaultdict(int))

for form in imqcam_builds:
    igsn = form['data'].get('assignedIGSN', 'UNKNOWN_IGSN')
    geometries = form['data'].get('buildGeometries', [])
    for geom in geometries:
        geometry_type = geom.get('geometryType')
        count = geom.get('count', 0)
        igsn_geometry_counts[igsn][geometry_type] += count

print("▶️ Geometry Counts by IGSN:")
for igsn, geom_counts in igsn_geometry_counts.items():
    print(f"\n{igsn}:")
    for geom_type, count in geom_counts.items():
        print(f"  {geom_type}: {count}")

# Part 2: Total geometry counts
total_geometry_counts = defaultdict(int)

for geom_counts in igsn_geometry_counts.values():
    for geom_type, count in geom_counts.items():
        total_geometry_counts[geom_type] += count

# Print total geometry counts
print("\n🧮 Total Geometry Counts Across All IGSNs:")
for geom_type, total in total_geometry_counts.items():
    print(f"{geom_type}: {total}")

▶️ Geometry Counts by IGSN:

CMXMAL00007:
  TEN: 30
  AXFT: 20

CMXMAL00008:
  4PTF: 32

CMXMAL00010:
  4PTF: 32

🧮 Total Geometry Counts Across All IGSNs:
TEN: 30
AXFT: 20
4PTF: 64


In [7]:
from collections import defaultdict
import json

infill_rows = []
# Map: geometryType -> role -> list of parameter dicts
geometry_param_summary = defaultdict(lambda: defaultdict(list))

for form in imqcam_builds:
    geometries = form['data'].get('buildGeometries', [])
    build_params = form['data'].get('buildParameters', {})

    for geom in geometries:
        geometry_type = geom.get('geometryType', 'UNKNOWN')
        for role, params in build_params.items():
            geometry_param_summary[geometry_type][role].append(params)
            if role == 'infillParameters':
                row = {
                    'geometryType': geometry_type,
                    **params
                }
                infill_rows.append(row)

print("▶️ Build Parameters by Geometry Type:\n")

from collections import Counter

def serialize_params(param_dict):
    return json.dumps(param_dict, sort_keys=True)

for geom_type, role_dict in geometry_param_summary.items():
    print(f"Geometry: {geom_type}")
    for role, param_list in role_dict.items():
        print(f"  Role: {role}")
        counts = Counter(serialize_params(p) for p in param_list)
        for param_json, count in counts.items():
            print(f"    Params: {json.loads(param_json)} -> {count} builds")
    print()

▶️ Build Parameters by Geometry Type:

Geometry: TEN
  Role: contouringParameters
    Params: {'hatchSpacing': 0, 'laserPower': 150, 'laserVelocity': 1250, 'layerThickness': 30} -> 1 builds
  Role: downskinParameters
    Params: {'hatchSpacing': 0.08, 'laserPower': 90, 'laserVelocity': 1200, 'layerThickness': 30} -> 1 builds
  Role: infillParameters
    Params: {'hatchSpacing': 0.14, 'laserPower': 350, 'laserVelocity': 1300, 'layerThickness': 30} -> 1 builds
  Role: upskinParameters
    Params: {'hatchSpacing': 0, 'laserPower': 0, 'laserVelocity': 0, 'layerThickness': 0} -> 1 builds

Geometry: AXFT
  Role: contouringParameters
    Params: {'hatchSpacing': 0, 'laserPower': 150, 'laserVelocity': 1250, 'layerThickness': 30} -> 1 builds
  Role: downskinParameters
    Params: {'hatchSpacing': 0.08, 'laserPower': 90, 'laserVelocity': 1200, 'layerThickness': 30} -> 1 builds
  Role: infillParameters
    Params: {'hatchSpacing': 0.14, 'laserPower': 350, 'laserVelocity': 1300, 'layerThickness': 

In [8]:
imqcam_builds_infill = pd.DataFrame(infill_rows)
print("\n📊 DataFrame of infillParameters builds:")

# convertion from microns to mm for comformity with other builds
imqcam_builds_infill['hatchSpacing'] /= 1000
imqcam_builds_infill['layerThickness'] /= 1000

print(imqcam_builds_infill.head())


📊 DataFrame of infillParameters builds:
  geometryType  hatchSpacing  laserPower  laserVelocity  layerThickness
0          TEN       0.00014         350           1300            0.03
1         AXFT       0.00014         350           1300            0.03
2         4PTF       0.14000         370           1200            0.03
3         4PTF       0.14000         370           1200            0.03


# TTT

In [9]:
ttt_builds[0]

{'_id': '67eaf380366ec49ab59ddbb0',
 'created': '2025-03-31T19:56:48.906000+00:00',
 'data': {'Build_Date': '2022-01-01',
  'Build_ID': 'A01_2022-01-01_JHUAPL_EOSM290_Ti64_Cylinder',
  'DOE_Code': 1,
  'DOE_ID': 'A',
  'Elongation_Percent': 7.329018,
  'Geometry': 'Cylinder',
  'Hatch_mm': 0.14,
  'Layer_mm': 0.03,
  'Location': 'JHUAPL_EOSM290',
  'Material': 'Ti64',
  'Modulus_GPa': 82.95913,
  'Orientation': 'Horizontal',
  'Parameter_Label': 'A01',
  'Peak_Stress_MPa': 1047.749,
  'Percent_Porosity_Contour': 0.0109683,
  'Porosity_percent_infill': 0.312183,
  'Power_W': 160,
  'Specimen_Position': 'On Build Plate',
  'Speed_mm_per_s': 1200,
  'Total_Porosity_Percent': 0.245827,
  'VED_J_per_mm3': 31.74603174603174,
  'Yield_Strength_MPa': 877.268},
 'files': [],
 'folderId': '67d394ae366ec49ab59dd4dd',
 'folders': [],
 'formId': '67d39472366ec49ab59dd4db',
 'uniqueId': 'A01_2022-01-01_JHUAPL_EOSM290_Ti64_Cylinder',
 'updated': '2025-03-31T19:56:48.906000+00:00'}

In [12]:
ttt_rows = []

for form in ttt_builds:
    d = form.get('data', {})

    row = {
        'geometryType': 'TTT',  # Hardcoded since geometry is missing
        'hatchSpacing': d.get('Hatch_mm'),
        'laserPower': d.get('Power_W'),
        'laserVelocity': d.get('Speed_mm_per_s'),
        'layerThickness': d.get('Layer_mm')
    }

    ttt_rows.append(row)

ttt_builds_infill = pd.DataFrame(ttt_rows)

# Append to existing df_infill
df_infill = pd.concat([imqcam_builds_infill, ttt_builds_infill], ignore_index=True)

print("📊 DataFrame for IMQCAM and TTT builds:")
print(df_infill.tail())

📊 DataFrame for IMQCAM and TTT builds:
    geometryType  hatchSpacing  laserPower  laserVelocity  layerThickness
200          TTT          0.11         277           2641            0.03
201          TTT          0.09          62            444            0.03
202          TTT          0.05         106           1736            0.03
203          TTT          0.11          60            539            0.03
204          TTT          0.18         141            486            0.03


# ULI

In [13]:
uli_builds[0]

{'_id': '689232cd6b3eda3a4033ad6a',
 'created': '2025-08-05T16:35:25.503000+00:00',
 'creatorId': '65b80ef4f4d3724f8bc3a004',
 'data': {'buildId': 'CMU01.01',
  'depositionId': '688bb89d98483efffe7fd1d2',
  'hatchSpacing': 0.14,
  'location': 'CMU',
  'lookup': 'CMXMAL00011-01 - 688bb89d98483efffe7fd1d2 - CMU01.01',
  'projectName': 'ULI',
  'scanPower': 370,
  'scanVelocity': 950},
 'files': [],
 'folderId': None,
 'folders': [],
 'formId': '68922e35f5b193b7d3e07f5b',
 'uniqueId': 'CMU01.01',
 'updated': '2025-08-05T16:35:25.503000+00:00'}

In [14]:
uli_rows = []

for form in uli_builds:
    d = form.get('data', {})

    row = {
        'geometryType': 'ULI',
        'hatchSpacing': d.get('hatchSpacing'),
        'laserPower': d.get('scanPower'),
        'laserVelocity': d.get('scanVelocity'),
        'layerThickness': None  # Explicitly set to None
    }

    uli_rows.append(row)

uli_builds_infill = pd.DataFrame(uli_rows)

df_infill = pd.concat([df_infill, uli_builds_infill], ignore_index=True)

print("📊 Final Combined DataFrame:")
print(df_infill.tail())

📊 Final Combined DataFrame:
    geometryType  hatchSpacing laserPower  laserVelocity layerThickness
392          ULI          0.14        280         1050.0           None
393          ULI          0.14        280         1350.0           None
394          ULI          0.14        280         1200.0           None
395          ULI          0.14        280         1050.0           None
396          ULI          0.14        280         1150.0           None


In [ ]:
# Dump to CSV
df_infill.to_csv("all_infill.csv", index=False)